<a href="https://colab.research.google.com/github/Komal77rao/Data-Eng-Modules/blob/main/2-object-relations/11-object-relations/index-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object Relations

### Introduction

Now so far we have worked with individual classes, but in a larger program, we'll need to create multiple classes, whose instances interact with one another.  Let's see
how.

### Storing our instances

Let's move back to our plane example.  We can create a new plane with something like the following.

In [1]:
import datetime
class Plane:
    def __init__(self, year):
        self.year = year

    def age(self):
        now = datetime.datetime.now() #local variable
        current_year = now.year
        return current_year - self.year

Now, in the life of our program, we may mant to create multiple planes, and after the plane is created may want an easy way to reference each of the planes that are created.  To do this, let's create a `store` object that looks like the following.

In [2]:
store = {'planes': []}

Now inside this dictionary, we can store each of the planes that are created.  Now, we can do this simply by creating a new plan, and then adding it to the store.

In [3]:
plane = Plane(2010)

In [4]:
store['planes'].append(plane)

In [5]:
store

{'planes': [<__main__.Plane at 0x7b6279f10670>]}

However, to ensure that this occurs automatically, upon creating a plan, we should add this process into the `__init__` function like so.

In [8]:
store = {'planes': []}

class Plane:
    def __init__(self, year):
        # *** line we are adding ***
        store['planes'].append(self)
        self.year = year

    def age(self):
        now = datetime.datetime.now()
        current_year = now.year
        return current_year - self.year

So this time, each time a plan is created, we automatically add it to the store.

> Let's try this by creating two new planes.

In [10]:
Plane(2010)
Plane(2011)

store

{'planes': [<__main__.Plane at 0x7b6279f10280>,
  <__main__.Plane at 0x7b6279f12650>]}

What's nice about this is that if we say, would like to see all of the years that our planes were made, we can do so with the following:

In [11]:
planes = store['planes']
[plane.year for plane in planes]

[2010, 2011, 2010, 2011]

So saving our instances in a collection is a nice way to see aggregate information about all of the planes created.

Let's give each of these planes an id as well.  We can do so by updating our code to the following.

In [12]:
store = {'planes': []}

class Plane:
    def __init__(self, year):
        self.year = year
        # plane counter
        plane_id = len(store['planes']) + 1
        store['planes'].append(self)
        self.id = plane_id

    def age(self):
        now = datetime.datetime.now()
        current_year = now.year
        return current_year - self.year

In [24]:
Plane(2009)
Plane(2010)
Plane(2011)

In [23]:
[plane.__dict__
 for plane in store['planes']]

[{'year': 2009, 'id': 1},
 {'year': 2010, 'id': 2},
 {'year': 2011, 'id': 3},
 {'year': 2015, 'id': 4},
 {'year': 2009, 'id': 5},
 {'year': 2010, 'id': 6},
 {'year': 2011, 'id': 7},
 {'year': 2009, 'id': 8},
 {'year': 2010, 'id': 9},
 {'year': 2011, 'id': 10},
 {'year': 2009, 'id': 11},
 {'year': 2010, 'id': 12},
 {'year': 2011, 'id': 13}]

In [20]:
plane = Plane(2015)

plane.id

4

And now, this time, each of the planes has their own id.

### Adding flights

Now let's say we want to keep track of flights.  We can do so with something like the following.

In [15]:
store = {'planes': [], 'flights': []}

class Flight:
    def __init__(self, origin_city, destination_city):
        store['flights'].append(self)
        self.id = len(store['flights'])
        self.origin_city = origin_city
        self.destination_city = destination_city

In [16]:
flight_1 = Flight('NYC', 'CHI')
flight_2 = Flight('NYC', 'LA')

And then we can look reference each of the flights created with something like the following.

In [17]:
store['flights']

[<__main__.Flight at 0x7b6279f11420>, <__main__.Flight at 0x7b6279f10610>]

In [18]:
[flight.__dict__ for flight in store['flights']]

[{'id': 1, 'origin_city': 'NYC', 'destination_city': 'CHI'},
 {'id': 2, 'origin_city': 'NYC', 'destination_city': 'LA'}]

Now it probably makes sense to associate a flight with a plane.  If we were to describe the relationship between flights and planes, we would say that:

* A flight **has one** plane
* A plane **has many** flights

> Take a moment to think about the above.  It's one of the key relationships we'll see.  A flight can only have one particular plane associated with it, but a plane can have many associated flights.

Ok, now to link a a flight to a plane in code we can do something like the following.

In [28]:
class Flight:
    def __init__(self, origin_city, destination_city, plane):

        self.id = len(store['flights']) + 1
        self.origin_city = origin_city
        self.destination_city = destination_city

        self.plane_id = plane.id
        store['flights'].append(self)

    def plane(self):
        return [plane for plane in store['planes'] if plane.id == flight.plane_id][0]

In [32]:
[flight.__dict__ for flight in store['flights']]
[plane.__dict__ for plane in store['planes']]

[{'year': 2009, 'id': 1},
 {'year': 2010, 'id': 2},
 {'year': 2011, 'id': 3},
 {'year': 2015, 'id': 4},
 {'year': 2009, 'id': 5},
 {'year': 2010, 'id': 6},
 {'year': 2011, 'id': 7},
 {'year': 2009, 'id': 8},
 {'year': 2010, 'id': 9},
 {'year': 2011, 'id': 10},
 {'year': 2009, 'id': 11},
 {'year': 2010, 'id': 12},
 {'year': 2011, 'id': 13},
 {'year': 2009, 'id': 14},
 {'year': 2010, 'id': 15},
 {'year': 2011, 'id': 16}]

In [33]:
store = {'planes': [], 'flights': []}

plane = Plane(2015)

In [34]:
flight = Flight('NYC', 'CHI', plane)

In [36]:
flight.plane().__dict__

{'year': 2015, 'id': 1}

Let's break down what we did above.  First we stored the `plane_id` on our flight with the line `self.plane_id = plane.id` in our `__init__` function.  

In [ ]:
flight.plane_id

Then in the `plane` function, to return the associated plane, we search through the associated planes until we find the plane whose id matches the flight's `plane_id`.

```python
[plane for plane in store['planes'] if plane.id == flight.plane_id][0]
```

### Adding a has many function

So above, we saw how we can write a function to find the plane that belongs to the flight.  Now what if we want to find *all* of the flights associated with a plane.

Let's start by creating another flight.

In [37]:
store = {'planes': [], 'flights': []}
plane = Plane(2015)
plane_two = Plane(2018)

flight = Flight('NYC', 'CHI', plane)
flight_two = Flight('NYC', 'CHI', plane_two)
flight_three = Flight('NYC', 'LA', plane)

So we want to find all of the flights associated with our plane.  Notice that on our plane, there is no associating data to the flight.

In [38]:
plane.__dict__

{'year': 2015, 'id': 1}

Rather, the link between planes and flights lives on the object that *has one*, that is the flight (as the flight has one plane).

In [39]:
[flight.__dict__ for flight in store['flights']]

[{'id': 1, 'origin_city': 'NYC', 'destination_city': 'CHI', 'plane_id': 1},
 {'id': 2, 'origin_city': 'NYC', 'destination_city': 'CHI', 'plane_id': 2},
 {'id': 3, 'origin_city': 'NYC', 'destination_city': 'LA', 'plane_id': 1}]

So to find all of the flights associated with the plane with id 1, we can do the following.

In [40]:
class Plane:
    def __init__(self, year):
        self.year = year
        self.id = len(store['planes']) + 1
        store['planes'].append(self)

    def age(self):
        now = datetime.datetime.now()
        current_year = now.year
        return current_year - self.year

    def flights(self):
        return [flight for flight in store['flights'] if flight.plane_id == self.id]

So our flights method goes through each of the flights, returning only the flight that has the plane_id that matches the current instances id.

Now let's try it by recreating some data.

In [42]:
store = {'planes': [], 'flights': []}
plane = Plane(2015)
plane_two = Plane(2018)

flight = Flight('NYC', 'CHI', plane)
flight_two = Flight('NYC', 'CHI', plane_two)
flight_three = Flight('NYC', 'LA', plane)

In [43]:
plane.id

1

So each plane_id is stored on the flight, and to find all of the flights associated with a plane, we call the flights method, which looks for the flights with the matching `plane_id`.

In [44]:
plane.flights()

[<__main__.Flight at 0x7b62659ac340>, <__main__.Flight at 0x7b62659ad420>]

In [45]:
[flight.__dict__ for flight in plane.flights()]

[{'id': 1, 'origin_city': 'NYC', 'destination_city': 'CHI', 'plane_id': 1},
 {'id': 3, 'origin_city': 'NYC', 'destination_city': 'LA', 'plane_id': 1}]

### Summary

In this lesson, we learned how to associate objects with one another.  We first saw that we can store each of the instances created in a `store` where our store is a dictionary that has keys to store instances of each of the classes, as well as an associated counter.  Then, every time an instance is created we add the new instance to the store through the `__init__` function.

In [52]:
store = {'planes': []}

class Plane:
    def __init__(self, year):
        store['planes'].append(self)
        self.id = len(store['planes'])
        self.year = year

In [55]:
Plane(2011)

In [56]:
[plane.__dict__ for plane in store['planes']]

[{'id': 2, 'year': 2011}, {'id': 3, 'year': 2011}]

Then we saw how we can associate a flight with a plane by storing the `plane_id` on the flight.

In [60]:
store = {'planes': [], 'flights': []}

class Flight:
    def __init__(self, origin_city, destination_city, plane):
        self.id = len(store['flights']) + 1
        self.origin_city = origin_city
        self.destination_city = destination_city
        self.plane_id = plane.id
        store['flights'].append(self)

    def plane(self):
        return [plane for plane in store['planes'] if plane.id == flight.plane_id][0]

In [61]:
#plane = Plane(2018)
flight = Flight('NYC', 'CHI', plane)
flight

And we can find the plane associated with a flight by looping through each of the planes, finding the plane whose id matches the flight's `plane_id`.

In [ ]:
flight.plane()

And we can also find a plane's flights by  searching through all flights, returning the flight whose `plane_id` matches the current flight's `id`.

In [ ]:
class Plane:
    def __init__(self, year):
        store['planes'].append(self)
        self.id = len(store['planes']) + 1
        self.year = year

    def flights(self):
        return [flight for flight in store['flights'] if flight.plane_id == self.id]